# DeepXDE

In [ ]:
%%capture
!pip install pyfoil
!pip install deepxde
!pip install scikit-optimize
!pip3 install torch torchvision torchaudio
# !pip install tensorflow
# !export DDEBACKEND='tensorflow.compat.v1'

## Flow around profile (PiDeepONet)



In [ ]:
import  numpy as np, types, skopt, os, pyfoil ,  pandas  as pd, matplotlib.pyplot as plt, json
from scipy.spatial.distance import cdist

from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
from skopt.space import Real, Categorical, Integer
# from skopt.utils import use_named_arg

from pathlib import Path 
import torch 
from deepxde.backend import torch 
import deepxde as dde#,tensorflow as tf 
dde.backend.load_backend('pytorch')
# dde.backend.load_backend('pytorch')


import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.path as mpath
   
from PIL import Image 

In [ ]:
global rho; rho  = 1.0
global mu;  mu   = 0.02
global T;   T = 273 + 20
global R;   R = 8.341
from pde import *

### Incompressible

In [ ]:
def navier_stokes_incomp_deeponet(xy, uvp, aux):
    """
    System of PDEs to be minimized: incompressible Navier-Stokes equation in the
    continuum-mechanics based formulations.
    """
    u         = uvp[:, 0:1] # Функция тока
    v         = uvp[:, 1:2] # Функция тока
    p         = uvp[:, 2:3] # Давление
    # Производные компонент скорости
    u_x = dde.grad.jacobian(u, xy, i = 0, j = 0)
    u_y = dde.grad.jacobian(u, xy, i = 0, j = 1)
    v_x = dde.grad.jacobian(v, xy, i = 0, j = 0)
    v_y = dde.grad.jacobian(v, xy, i = 0, j = 1)
    u_xx = dde.grad.hessian(u, xy, i = 0, j = 0)
    u_yy = dde.grad.hessian(u, xy, i = 1, j = 1)
    v_xx = dde.grad.hessian(v, xy, i = 0, j = 0)
    v_yy = dde.grad.hessian(v, xy, i = 1, j = 1)
    # Производные давления
    p_x = dde.grad.jacobian(p, xy, j = 0)
    p_y = dde.grad.jacobian(p, xy, j = 1)
    # Уравненияя сохранения массы
    eq_1 = u_x + v_y
    #  Уравнение движения (уравнение Навье-Стокса в компоненте x)
    eq_2_1 = rho * (u * u_x + v * u_y) - (
        - p_x + mu*(u_xx + u_yy) 
        )
    #  Уравнение движения (уравнение Навье-Стокса в компоненте y)
    eq_2_2 = rho * (u * v_x + v * v_y) - (
        - p_y + mu*(v_xx + v_yy) 
        )

    return [eq_1, eq_2_1, eq_2_2]

#### Train

In [ ]:
# From file
numInternal         = 10000
numXmax             = 2500
numXmin             = 2500
numYmax             = 2500
numYmin             = 2500
# Make domain
inner_points_N      = 10000
outer_points_N      = 10000
farfield_points_N   = 2500
airfoil_points_N    = 2500

In [ ]:
# Чтение данных из CSV файла
df = pd.read_csv(os.path.join(os.getcwd(),'NACA0012', 'case.csv'))
# Open the JSON file in read mode
with open(os.path.join(os.getcwd(),'NACA0012', "caseInfo.json"), 'r') as file:
    # Load the JSON data from the file into a Python object
    infoCase = json.load(file)

xmin, ymin, xmax, ymax, UMax, VMax, pMax = [infoCase[name] for name in ['xmin', 'ymin', 'xmax', 'ymax', 'UMax', 'VMax', 'pMax']]
# Извлечение значений из DataFrame

U_selected = df['U'].values.reshape(-1, 1)
V_selected = df['V'].values.reshape(-1, 1)
P_selected = df['p'].values.reshape(-1, 1)
x_selected = df['x'].values.reshape(-1, 1)
y_selected = df['y'].values.reshape(-1, 1) 

xmin,xmax = x_selected.min(),x_selected.max(),
ymin,ymax = y_selected.min(),y_selected.max(),

In [ ]:
dde.config.set_random_seed(48)
dde.config.set_default_float('float32')

nacaParams = {
    'naca_code' : '0012',
    'c'         : 1,
    'offset_x'  : 0.0,
    'offset_y'  : 0.0,
    'angle_deg' : 0
}

from utils import *
foil_points = boundaryNACA(**nacaParams)

xmin_rec, xmax_rec = xmin + (-xmin + xmax)*1/4, xmin + (-xmin + xmax)*3/4
ymin_rec, ymax_rec = ymin + (-ymin + ymax)*1/4, ymin + (-ymin + ymax)*3/4

umax = UMax
vmax = VMax
global Re;  Re = rho * umax * nacaParams['c'] * 1 / mu

In [ ]:
farfield    = dde.geometry.Rectangle([xmin, ymin], [xmax, ymax])
airfoil     =   dde.geometry.Polygon(foil_points)

geom        = dde.geometry.CSGDifference(farfield, airfoil) 

inner_rec  = dde.geometry.Rectangle([xmin_rec, ymin_rec], [xmax_rec, ymax_rec])
outer_dom  = dde.geometry.CSGDifference(farfield, inner_rec)
outer_dom  = dde.geometry.CSGDifference(outer_dom, airfoil)
inner_dom  = dde.geometry.CSGDifference(inner_rec, airfoil)

inner_points = inner_dom.random_points(inner_points_N)
outer_points = outer_dom.random_points(outer_points_N)
airfoil_points = airfoil.random_points(airfoil_points_N)

farfield_points = farfield.random_boundary_points(farfield_points_N)

xy = np.hstack((x_selected, y_selected)) 
points = np.concatenate((
    inner_points,
    outer_points,
    farfield_points,
    foil_points,
    xy
    ), axis=0)

In [ ]:
def boundary_farfield_inlet(x, on_boundary):    return on_boundary and np.isclose(x[0], xmin)
def boundary_farfield_top_bottom(x, on_boundary):    return on_boundary and (np.isclose(x[1], ymax) or np.isclose(x[1], ymin))
def boundary_farfield_outlet(x, on_boundary):    return on_boundary and np.isclose(x[0], xmax)
def boundary_airfoil(x, on_boundary): return on_boundary and airfoil.on_boundary(np.array([x]))[0]

# Boundary values definition
def fun_u_farfield(x, y, _):   
    u         = y[:, 0:1] # Функция тока
    return u - umax
    
def fun_v_farfield(x, y, _):
    v         = y[:, 1:2] 
    return v - vmax
# foil
def fun_no_slip_u(x, y, _): 
    u         = y[:, 0:1] # Функция тока 
    return u  
def fun_no_slip_v(x, y, _): 
    v         = y[:, 1:2] # Функция тока   
    return v  

# outlet
def fun_outlet_u(x, y, _): 
    u = y[:, 0:1] # Функция тока 
    u_x = dde.grad.jacobian(u,x,j  = 0)      
    return u_x  
def fun_outlet_v(x, y, _): 
    v         = y[:, 1:2] # Функция тока 
    v_x = dde.grad.jacobian(v,x,j  = 0)   
    return v_x  
# top bottom
def fun_top_bottom_u(x, y, _): 
    u = y[:, 0:1] # Функция тока 
    u_y = dde.grad.jacobian(u,x,j  = 1)      
    return u_y  
def fun_top_bottom_v(x, y, _): 
    v         = y[:, 1:2] # Функция тока 
    v_y = dde.grad.jacobian(v,x,j  = 1)   
    return v_y  

In [ ]:
## U # м^(1)*с^(-2)
bc_inlet_u = dde.icbc.OperatorBC(geom, fun_u_farfield, boundary_farfield_inlet)
bc_top_bottom_u = dde.icbc.OperatorBC(geom, fun_top_bottom_u, boundary_farfield_top_bottom)
bc_outlet_u = dde.OperatorBC(geom,fun_outlet_u, boundary_farfield_outlet)
bc_airfoil_u = dde.icbc.OperatorBC(geom, fun_no_slip_u, boundary_airfoil)

bc_U = [bc_inlet_u,bc_outlet_u, bc_top_bottom_u, bc_airfoil_u]
observe_U = dde.icbc.PointSetBC(xy, U_selected, component=0)

## V # м^(1)*с^(-2)
bc_inlet_v = dde.OperatorBC(geom, fun_v_farfield, boundary_farfield_inlet)
bc_top_bottom_v = dde.OperatorBC(geom, fun_top_bottom_v, boundary_farfield_top_bottom)
bc_outlet_v = dde.OperatorBC(geom,fun_outlet_v, boundary_farfield_outlet)
bc_airfoil_v = dde.OperatorBC(geom, fun_no_slip_v, boundary_airfoil) 

bc_V = [bc_inlet_v,bc_outlet_v ,bc_top_bottom_v,bc_airfoil_v]
observe_V = dde.icbc.PointSetBC(xy, V_selected, component=1)

pout = 0 # м^(2)*с^(-2)
internalField_p_value = pout
# inlet farfield
def internalField_p(x,y,_):
  p = y[:, 2:3] # Давление 
  if dde.backend.backend_name=='tensorflow.compat.v1':
    return p - tf.cast(internalField_p_value, tf.float32)
  elif dde.backend.backend_name=='pytorch':
    return p - torch.tensor(internalField_p_value, dtype=torch.float32)
# foil
def zeroGradient_p(x,y,_):
  p = y[:, 2:3] # Давление 
  p_x = dde.grad.jacobian(p, x, j  = 0)
  p_y = dde.grad.jacobian(p, x,  j  = 1)
  return p_x + p_y
#  outlet
def outlet_p(x,y,_):
  p = y[:, 2:3] # Давление 
  p_x = dde.grad.jacobian(p, x, j  = 0)
  return p_x
# top bottom 
def top_bottom_p(x,y,_):
  p = y[:, 2:3] # Давление 
  p_y = dde.grad.jacobian(p, x,  j  = 1)
  return  p_y


bc_inlet_p = dde.icbc.OperatorBC(
    geom, internalField_p, boundary_farfield_inlet, #component=5
    )
bc_outlet_p = dde.icbc.OperatorBC(
    geom, outlet_p, boundary_farfield_outlet, #component=5
    )
bc_top_bottom_p = dde.icbc.OperatorBC(
    geom, top_bottom_p, boundary_farfield_top_bottom, #component=5
    )
bc_airfoil_p = dde.icbc.OperatorBC(
    geom, zeroGradient_p, boundary_airfoil,#component = 5
    )

bc_p = [bc_inlet_p,bc_outlet_p,bc_top_bottom_p,bc_airfoil_p]

observe_p = dde.icbc.PointSetBC(xy, P_selected, component=2)


In [ ]:
pde = dde.data.PDE(
    geom,
    navier_stokes_incomp_deeponet,
    [
        *bc_U,       observe_U, # 5
        *bc_V,       observe_V, # 10
        *bc_p,       observe_p, # 35
    ],
    num_domain=0,
    num_boundary=0,
    num_test=5000,
    anchors=points,
)

# Function space
# func_space = dde.data.GRF(length_scale=0.2)
func_space = dde.data.Chebyshev(N=100, M=1)

# Data
# Генерация равномерно распределенных точек в диапазоне [-1, 2] для x и [-1, 1] для y
n_pts_edge = 50
x_min, x_max = -1, 2
y_min, y_max = -1, 1
eval_pts_x = np.linspace(x_min, x_max, num=n_pts_edge)[:, None]
eval_pts_y = np.linspace(y_min, y_max, num=n_pts_edge)[:, None]

# Масштабирование точек x и y для приведения их к диапазону пространства Чебышева ([-1, 1])
scaled_eval_pts_x = (eval_pts_x - x_min) / (x_max - x_min) * 2 - 1
scaled_eval_pts_y = (eval_pts_y - y_min) / (y_max - y_min) * 2 - 1

# Создание общих точек eval_pts
eval_pts = np.concatenate((scaled_eval_pts_x, scaled_eval_pts_y), axis=1)

data = dde.data.PDEOperatorCartesianProd(
    pde, func_space, eval_pts, num_function=1000,
    function_variables=[0], num_test=100, batch_size=50
)

plt.figure(figsize=(32, 18))
plt.scatter(data.train_x[:, 0], data.train_x[:, 1], s=0.3)
plt.scatter(xy[:, 0], xy[:, 1], s=0.5, color='red')
plt.axis('equal')


In [ ]:
# Net
net = dde.nn.DeepONetCartesianProd(
    [n_pts_edge*2, 128, 128, 128],
    [2, 128, 128, 128],
    "tanh",
    "Glorot normal",
    num_outputs=3,
    multi_output_strategy="independent"
)

model_path = os.path.join(os.getcwd(),'model_DeepONet')
if not os.path.exists(model_path):
    os.makedirs(model_path)
    print(f"Папка '{model_path}' создана успешно.")
else:
    print(f"Папка '{model_path}' уже существует.")

# Model
model = dde.Model(data, net)
model.compile("adam", lr=0.001, decay=("inverse time", 10000, 0.5))

checker = dde.callbacks.ModelCheckpoint(
    os.path.join(model_path,'model.ckpt'),
    save_better_only=True,
    period=5000
)

train_story_path = os.path.join(os.getcwd(),'train_story_DeepONet')
if not os.path.exists(train_story_path):
    os.makedirs(train_story_path)
    print(f"Папка '{train_story_path}' создана успешно.")
else:
    print(f"Папка '{train_story_path}' уже существует.")

losshistory, train_state = model.train(
    iterations=100000, display_every = 5000,
    model_save_path = os.path.join(train_story_path,'train_story_DeepONet'),
    callbacks=[checker]
    )
dde.saveplot(losshistory, train_state, issave = True, isplot = True)

dde.utils.plot_loss_history(losshistory)